In [1]:
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

In [27]:
class Model(object):
    def __init__(self, name, dataset):
        self.name = name
        self.dataset = dataset
        self.is_training = False
        if not hasattr(self, 'rand_std'): self.rand_std = 0.030
    
    def __str__(self):
        return '{}/{}'.format(self.name, self.dataset)

    def exec_all(self, epoch_count=10, batch_size=10, learning_rate=0.001,
                 report=0, show_cnt=3, num_workers=0):
        self.train(epoch_count, batch_size, learning_rate, report, num_workers)
        self.test()
        if show_cnt > 0: self.visualize(show_cnt)

In [28]:
class Mlp_Torch(Model):
    def __init__(self, name, dataset, hconfigs):
        self.layers=[]
        super(Mlp_Torch, self).__init__(name, dataset)
        self.make_layers(hconfigs)

In [29]:
#우선 SLP 하나로 테스트 해보고 그 뒤 추후 늘린다.
def mlp_make_layer_torch(self, hconfigs):
    self.hconfig = hconfigs
    
    prev_shape = self.dataset.input_shape
    
    for hconfig in hconfigs:
        #hconfig에는 output값이 들어있음.
        _, prev_shape=self.alloc_make_layer(prev_shape, hconfig)
        
    output_cnt = int(np.prod(self.dataset.output_shape))
    self.alloc_make_layer(prev_shape, output_cnt, atf=False)

def mlp_alloc_make_layer_torch(self, input_shape, hconfig, atf=True):
    
    input_cnt = np.prod(input_shape)
    output_cnt = np.prod(hconfig)
    self.layers.append(nn.Linear(in_features=input_cnt,out_features=output_cnt))
    if atf:self.layers.append(nn.ReLU())
#     if atf==False and self.dataset.mode =='select':
#         self.layers.append(nn.Softmax(dim=1))
    return input_cnt, output_cnt
    
    
Mlp_Torch.make_layers = mlp_make_layer_torch
Mlp_Torch.alloc_make_layer = mlp_alloc_make_layer_torch

In [30]:
class Net(nn.Module):
    def __init__(self, layers):
        super(Net, self).__init__()
        
        self.layer1 = nn.Sequential(*layers)
        
    def forward(self, x):
        out = self.layer1(x)

        return out

In [3]:
def train_torch(self, epoch_count=10, batch_size=10, \
                    learning_rate=0.001, report=0, num_workers = 0):
    
    self.learning_rate = learning_rate
    time1 = time2 = int(time.time())
    
    if report != 0:
        print('Model {} train started'.format(self.name),'\n'*2)
    
    
    train_loader=self.dataset.dataloader(self.dataset.tr_xs, self.dataset.tr_ys, batch_size, num_workers)
    self.model=Net(self.layers).to(DEVICE)
    optimizer = torch.optim.SGD(self.model.parameters(),lr = self.learning_rate)
#     optimizer=torch.optim.Adam(self.model.parameters(),lr=self.learning_rate,betas=(0.9,0.999),eps=1e-08)
    if self.dataset.mode == 'regression':
        loss_func = nn.MSELoss()
        
    elif self.dataset.mode == 'binary':
        loss_func = nn.BCEWithLogitsLoss()
    
    elif self.dataset.mode == 'select':
        loss_func = nn.CrossEntropyLoss()
    print('!!!!!!!!!!!')    
    print('!! Layer !! .\n!!!!!!!!!!! \n\n',self.model)
    print('\n!!!!!!!!!!!!!!!')    
    print('!! optimizer !! \n!!!!!!!!!!!!!!! \n\n', optimizer)
    for epoch in range(epoch_count):
        costs = []
        accs = []
        
        for batch_idx, samples in enumerate(train_loader):

            X_train, y_train = samples
            
            X_train=X_train.to(DEVICE)
            y_train=y_train.to(DEVICE)
            
            optimizer.zero_grad()
            
            output = self.model.forward(X_train)
#             print(output.shape)
#             print(y_train.squeeze_().shape)
#             print(output,'output')
#             print(y_train.squeeze_(),'y_train')
            loss = loss_func(output, torch.argmax(y_train,dim=1))
            acc = self.eval_accuracy(X_train, y_train, output)
            
            loss.backward()
            optimizer.step()
            
            costs.append(loss.cpu().detach().numpy())
            accs.append(acc.cpu().detach().numpy())
#             print(output)

            if report > 0 and (batch_idx+1) % report == 0:
                va_X, va_Y = self.dataset.get_validate_date(100)
                acc = self.eval_accuracy(va_X, va_Y)
                time3 = int(time.time())
                tm1, tm2 = time3 - time2, time3 - time1
                self.dataset.train_prt_result(epoch+1, costs,accs,acc, tm1, tm2)
                time2 = time3
                
    tm_total = int(time.time()) - time1
    print('Model {} train ended in {} secs:'.format(self.name, tm_total))

            
Mlp_Torch.train = train_torch

NameError: name 'Mlp_Torch' is not defined

In [15]:
def test_torch(self):
    with torch.no_grad():
        test_X,test_y=self.dataset.get_test_data()
        time1 = int(time.time())        
        acc = self.eval_accuracy(test_X, test_y)
        time2 = int(time.time())
        self.dataset.test_prt_result(self.name, acc, time2-time1)

Mlp_Torch.test = test_torch

In [ ]:
def mlp_eval_accuracy_torch(self, x, y, output=None):
    if output is None:
        with torch.no_grad():
            output = self.model.forward(x)
    accuracy = self.dataset.eval_accuracy(x, y, output)
    
    return accuracy

Mlp_Torch.eval_accuracy = mlp_eval_accuracy_torch

In [ ]:
def mlp_get_estimate_torch(self, x):
    with torch.no_grad():
        output = self.model.forward(x)
    estimate = self.dataset.get_estimate(output)
    return estimate

Mlp_Torch.get_estimate = mlp_get_estimate_torch

In [ ]:
def mlp_model_visualize_torch(self, num):
    print('Model {} Visualization'.format(self.name))
    deX, deY = self.dataset.get_visualize_data(num)
    est = self.get_estimate(deX)
    self.dataset.visualize(deX, est, deY)

Mlp_Torch.visualize = mlp_model_visualize_torch

In [15]:
# a=torch.tensor([[0.3,0.3,0.4],
# [0.4,0.3,0.3]])
# b=torch.tensor([[0.,0.,1.],
#                [1.,0.,0.]])
# print(a.shape, b.shape)
# nn.CrossEntropyLoss(a,b)

torch.Size([2, 3]) torch.Size([2, 3])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous